In [1]:
import sys
# if there are no conflicting packages in the default Python Libs =>
sys.path.append("/usr/home/username/pdfminer")

In [6]:
import spacy
from spacy.matcher import Matcher
import os
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2

In [ ]:
import spacy
from spacy.util import minibatch

nlp = spacy.load('en_core_web_sm') 
ner = nlp.get_pipe("ner")  # Access the NER component

for label in ["SKILL", "EDUCATION", "EXPERIENCE", ...]:
    ner.add_label(label)  # Add your custom labels

optimizer = nlp.initialize()  

for itn in range(100):  # Adjust number of iterations
    losses = {}
    batches = minibatch(training_data, size=8) # Adjust batch size
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, sgd=optimizer, losses=losses)
nlp.to_disk("resume_parser_model")

In [9]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        parser = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(parser.pages)):
            page = parser.pages[page_num]
            text += page.extract_text()
    return text

def process_resume(doc):
    skills = [ent.text for ent in doc.ents if ent.label_ in ["SKILLS","Skills","skills"]]
    experience = [ent.text for ent in doc.ents if ent.label_ in ["EXPERIENCE","Experience","experience"]]
    # ... extract other relevant sections
    return {"skills": skills, "experience": experience} 

def calculate_similarity(resume_data, job_desc):
    vectorizer = TfidfVectorizer()
    resume_text = " ".join([
        " ".join(resume_data["skills"]),
        " ".join(resume_data["experience"])
    ])
    tfidf_matrix = vectorizer.fit_transform([resume_text, job_desc])
    return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

In [16]:
# -- Main Logic --
nlp = spacy.load("resume_parser_model") 
job_description = "Information technology specialist"

resume_folder = "/Users/sreehariguruprasad/myprojects/AI Project/resume dataset/data/data/INFORMATION-TECHNOLOGY"
all_resumes = [f for f in os.listdir(resume_folder) if f.endswith(".pdf")]
random.shuffle(all_resumes)  # Shuffle for random selection

test_size = 0.2  # 20% of the dataset for testing
test_set_count = int(len(all_resumes) * test_size)
test_set = all_resumes[:test_set_count]
training_set = all_resumes[test_set_count:]

# -- Training --
selected_resumes = random.sample(training_set, 10)

for resume_file in selected_resumes:
    file_path = os.path.join(resume_folder, resume_file)
    text = extract_text_from_pdf(file_path)
    doc = nlp(text)
    data = process_resume(doc)
    similarity = calculate_similarity(data, job_description)
    print(f"{resume_file} - Similarity: {similarity:.2f}")

# Suggest the resume with the highest similarity

# -- Testing --
for resume_file in test_set:
    file_path = os.path.join(resume_folder, resume_file)
    text = extract_text_from_pdf(file_path)
    doc = nlp(text)
    data = process_resume(doc)
    similarity = calculate_similarity(data, job_description)
    print(f"[Test] {resume_file} - Similarity: {similarity:.2f}")

OSError: [E050] Can't find model 'resume_parser_model'. It doesn't seem to be a Python package or a valid path to a data directory.